In [1]:
import os 
import sys
import glob
import time
from tqdm import tqdm

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 


import MDAnalysis as mda 
from MDAnalysis.analysis import rms
from MDAnalysis.analysis import align

In [2]:
runs = sorted(glob.glob("../../traj_save/*//*.pdb"))

In [3]:
df = []

for run in runs[:]:
    pdb_file = run
    dcd_file = run.replace("pdb", "dcd")
    ref_pdb = pdb_file
    
    run_base = os.path.basename(run)[:-4]
    run_label = run_base
    if run.split('/')[-2] == 'Nsp10_Nsp16': 
        run_base += '_000'
#     print(run_base)
#     continue 
    
    mda_u = mda.Universe(pdb_file, dcd_file)
    protein = mda_u.select_atoms("protein")
#     calphas = mda_u.select_atoms("name CA") 

    for seg in protein.segments: 
        segid = seg.segid
        selection = f'name CA and segid {segid}'
        prealigner = align.AlignTraj(mda_u, mda_u, select=selection, in_memory=True).run()
        reference_coordinates = mda_u.trajectory.timeseries(asel=protein).mean(axis=1)
        reference = mda.Merge(protein).load_new(
                reference_coordinates[:, None, :], order="afc")

        aligner = align.AlignTraj(mda_u, reference, select=selection, in_memory=True).run()

        calphas = mda_u.select_atoms(selection) 
        rmsfer = rms.RMSF(calphas, verbose=True).run()

        for atom_ca, rmsf_v in zip(calphas, rmsfer.rmsf):
            chain = 'nsp10' if atom_ca.resnum < 5000 else 'nsp16'
            row = {
                'sys_name': run_base, 
#                 'Cation': sol_cation,
                'resname': atom_ca.resname, 
                'resnum': atom_ca.resnum, 
                'chain': chain, # atom_ca.segid, 
                'rmsf': rmsf_v
                }
            df.append(row)
        

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [5]:
df = pd.DataFrame(df)
df.head()

,sys_name,resname,resnum,chain,rmsf
0,comp_0_000,ALA,6798,nsp16,9.228327
1,comp_0_000,SER,6799,nsp16,7.519331
2,comp_0_000,SER,6800,nsp16,5.815448
3,comp_0_000,GLN,6801,nsp16,3.933446
4,comp_0_000,ALA,6802,nsp16,2.563892


In [6]:
df.to_pickle('rmdf_nsp10-16_all.pkl')